In [3]:
import pandas as pd
import numpy as np
import os

os.listdir('./data')

['TCGA_data.csv', 'urine_k15_0419.csv']

In [202]:
TCGA = pd.read_csv('./data/TCGA_data.csv')
urine = pd.read_csv('./data/urine_k15_0420.csv')
#urine = urine.drop(['Unnamed: 0','X'], axis = 1)
#urine.to_csv('./data/urine_K15_0420.csv', index = False)

In [203]:
TCGA.y.value_counts()

Normal    1706
KIRC       528
PRAD       492
BLCA       408
KIRP       288
KICH        66
Name: y, dtype: int64

In [204]:
print('TCGA shape', TCGA.shape)
print('urine shape', urine.shape)

TCGA shape (3488, 787)
urine shape (104, 787)


In [205]:
TCGA = TCGA[(TCGA.y == 'BLCA') | (TCGA.y == 'Normal')]
TCGA = TCGA.replace(['BLCA','Normal'],[ 1,0 ])
TCGA = TCGA.reset_index(drop = True)

In [206]:
#TCGA.to_csv('./data/TCGA_binary.csv', index = False)

In [207]:
TCGA.shape

(2114, 787)

In [208]:
TCGA.y.value_counts()

0    1706
1     408
Name: y, dtype: int64

In [209]:
#NA 없음
TCGA.isnull().sum().sum()

0

In [210]:
urine.y.value_counts()

BLCA_15k    42
NL_15k      21
RCC_100k    21
PRAD_15k    20
Name: y, dtype: int64

In [211]:
urine = urine[(urine.y == 'BLCA_15k') | (urine.y == 'NL_15k')]
urine = urine.replace(['BLCA_15k','NL_15k'],[ 1,0 ])
urine = urine.reset_index(drop = True)

In [212]:
urine.y.value_counts()

1    42
0    21
Name: y, dtype: int64

In [213]:
#urine.to_csv('./data/urine_binary.csv', index = False)

In [214]:
X = TCGA.drop('y', axis = 1)
y = TCGA['y']

In [154]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve

def get_eval(y_test, pred) :
    
    print('accuracy : {0:.4f} precision : {1:.4f} recall : {2:.4f} F1 : {3:.4f} AUC : {4:.4f}'.format(
                                                        accuracy_score(y_test, pred), 
                                                        precision_score(y_test, pred),
                                                        recall_score(y_test, pred),
                                                        f1_score(y_test,pred),
                                                        roc_auc_score(y_test, pred)
                                                                      ))
    
    
    
    #print('accuracy : {0:.4f}'.format(accuracy_score(y_test,pred)), 'roc_auc_score : ', roc_auc_score(y_test,pred))
    #print(classification_report(y_test, pred))
    print(confusion_matrix(y_test, pred))

In [125]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 5, random_state=0, shuffle = True)

# logistic regression

In [168]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler

standardsc = StandardScaler()
mms = MinMaxScaler()
mas = MaxAbsScaler()

lr = LogisticRegression()
import warnings
warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline

In [179]:
def model_pipeline(model) :
    pipeline1 = Pipeline([('StandardScaler', standardsc), ('Logistic', model)])
    pipeline2 = Pipeline([('MinMaxScaler', mms), ('Logistic', model)])
    pipeline3 = Pipeline([('MaxAbsScaler', mas), ('Logistic', model)])

    cv_acc = []
    pip1_acc = []
    pip2_acc = []
    pip3_acc = []

    for i, (tr_ind, te_ind) in enumerate(skf.split(X,y)) :

        X_train, X_test = X.iloc[tr_ind], X.iloc[te_ind]
        y_train, y_test = y[tr_ind], y[te_ind]

        model.fit(X_train, y_train)
        pipeline1.fit(X_train, y_train)
        pipeline2.fit(X_train, y_train)
        pipeline3.fit(X_train, y_train)

        cv_acc.append(accuracy_score(y[te_ind], model.predict(X_test)))
        pip1_acc.append(accuracy_score(y[te_ind], pipeline1.predict(X_test)))
        pip2_acc.append(accuracy_score(y[te_ind], pipeline2.predict(X_test)))
        pip3_acc.append(accuracy_score(y[te_ind], pipeline3.predict(X_test)))

        print('{0} 번째 accuracy non_scale : {1:.4f}% StandardScale : {2:.4f}% MinMax : {3:.4f}% MaxAbs : {4:.4f}%'.format(i,cv_acc[i],pip1_acc[i],pip2_acc[i],pip3_acc[i]))

    print('\n mean accuracy non_scale : {0:.4f}% StandardScale : {1:.4f}% MinMax : {2:.4f}% MaxAbs : {3:.4f}%'.format(np.mean(cv_acc),np.mean(pip1_acc),np.mean(pip2_acc),np.mean(pip3_acc)))

In [180]:
model_pipeline(lr)

0 번째 accuracy non_scale : 0.9788% StandardScale : 0.9764% MinMax : 0.8137% MaxAbs : 0.9741%
1 번째 accuracy non_scale : 0.9905% StandardScale : 0.9882% MinMax : 0.8132% MaxAbs : 0.9905%
2 번째 accuracy non_scale : 0.9787% StandardScale : 0.9787% MinMax : 0.8132% MaxAbs : 0.9716%
3 번째 accuracy non_scale : 0.9858% StandardScale : 0.9858% MinMax : 0.8128% MaxAbs : 0.9834%
4 번째 accuracy non_scale : 0.9763% StandardScale : 0.9787% MinMax : 0.8128% MaxAbs : 0.9739%

 mean accuracy non_scale : 0.9820% StandardScale : 0.9816% MinMax : 0.8131% MaxAbs : 0.9787%


In [181]:
pipeline1.fit(X,y)
get_eval(urine['y'],pipeline1.predict(urine.drop('y', axis = 1)))

accuracy : 0.6984 precision : 1.0000 recall : 0.5476 F1 : 0.7077 AUC : 0.7738
[[21  0]
 [19 23]]


# randomforest

In [182]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [183]:
model_pipeline(rf)

0 번째 accuracy non_scale : 0.9741% StandardScale : 0.1934% MinMax : 0.1934% MaxAbs : 0.9764%
1 번째 accuracy non_scale : 0.9882% StandardScale : 0.5177% MinMax : 0.1939% MaxAbs : 0.9858%
2 번째 accuracy non_scale : 0.9787% StandardScale : 0.6052% MinMax : 0.1939% MaxAbs : 0.9764%
3 번째 accuracy non_scale : 0.9858% StandardScale : 0.3934% MinMax : 0.1919% MaxAbs : 0.9858%
4 번째 accuracy non_scale : 0.9858% StandardScale : 0.9716% MinMax : 0.1919% MaxAbs : 0.9787%

 mean accuracy non_scale : 0.9825% StandardScale : 0.5363% MinMax : 0.1930% MaxAbs : 0.9806%


In [184]:
rf.fit(X,y)
get_eval(urine['y'],rf.predict(urine.drop('y', axis = 1)))

accuracy : 0.9524 precision : 1.0000 recall : 0.9286 F1 : 0.9630 AUC : 0.9643
[[21  0]
 [ 3 39]]


# adaboost

In [185]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()

In [186]:
model_pipeline(ada)

0 번째 accuracy non_scale : 0.9788% StandardScale : 0.8703% MinMax : 0.1934% MaxAbs : 0.9858%
1 번째 accuracy non_scale : 0.9905% StandardScale : 0.9811% MinMax : 0.1939% MaxAbs : 0.9882%
2 번째 accuracy non_scale : 0.9693% StandardScale : 0.9740% MinMax : 0.1939% MaxAbs : 0.9716%
3 번째 accuracy non_scale : 0.9787% StandardScale : 0.2133% MinMax : 0.1919% MaxAbs : 0.9787%
4 번째 accuracy non_scale : 0.9882% StandardScale : 0.9739% MinMax : 0.1919% MaxAbs : 0.9834%

 mean accuracy non_scale : 0.9811% StandardScale : 0.8025% MinMax : 0.1930% MaxAbs : 0.9815%


In [187]:
pipeline3 = Pipeline([('MaxAbsScaler', mas), ('AdaBoostClassifier', ada)])
pipeline3.fit(X,y)
get_eval(urine['y'],pipeline3.predict(urine.drop('y', axis = 1)))

accuracy : 0.9365 precision : 1.0000 recall : 0.9048 F1 : 0.9500 AUC : 0.9524
[[21  0]
 [ 4 38]]


# decision tree

In [188]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()

In [189]:
model_pipeline(tree)

0 번째 accuracy non_scale : 0.9670% StandardScale : 0.1580% MinMax : 0.1934% MaxAbs : 0.9646%
1 번째 accuracy non_scale : 0.9551% StandardScale : 0.9409% MinMax : 0.1939% MaxAbs : 0.9740%
2 번째 accuracy non_scale : 0.9456% StandardScale : 0.1939% MinMax : 0.1939% MaxAbs : 0.9811%
3 번째 accuracy non_scale : 0.9123% StandardScale : 0.1635% MinMax : 0.1919% MaxAbs : 0.9645%
4 번째 accuracy non_scale : 0.9431% StandardScale : 0.2014% MinMax : 0.1919% MaxAbs : 0.9645%

 mean accuracy non_scale : 0.9446% StandardScale : 0.3315% MinMax : 0.1930% MaxAbs : 0.9697%


In [190]:
pipeline3 = Pipeline([('MaxAbsScaler', mas), ('tree', tree)])
pipeline3.fit(X,y)
get_eval(urine['y'],pipeline3.predict(urine.drop('y', axis = 1)))

accuracy : 0.9524 precision : 1.0000 recall : 0.9286 F1 : 0.9630 AUC : 0.9643
[[21  0]
 [ 3 39]]


# GradientBoostingClassifier

In [191]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()

In [192]:
model_pipeline(gb)

0 번째 accuracy non_scale : 0.9835% StandardScale : 0.9741% MinMax : 0.1934% MaxAbs : 0.9858%
1 번째 accuracy non_scale : 0.9835% StandardScale : 0.9551% MinMax : 0.1939% MaxAbs : 0.9882%
2 번째 accuracy non_scale : 0.9764% StandardScale : 0.9787% MinMax : 0.1939% MaxAbs : 0.9764%
3 번째 accuracy non_scale : 0.9810% StandardScale : 0.9834% MinMax : 0.1919% MaxAbs : 0.9858%
4 번째 accuracy non_scale : 0.9787% StandardScale : 0.9716% MinMax : 0.1919% MaxAbs : 0.9858%

 mean accuracy non_scale : 0.9806% StandardScale : 0.9726% MinMax : 0.1930% MaxAbs : 0.9844%


In [193]:
pipeline3 = Pipeline([('MaxAbsScaler', mas), ('GB', gb)])
pipeline3.fit(X,y)
get_eval(urine['y'],pipeline3.predict(urine.drop('y', axis = 1)))

accuracy : 1.0000 precision : 1.0000 recall : 1.0000 F1 : 1.0000 AUC : 1.0000
[[21  0]
 [ 0 42]]


# SVM

In [198]:
from sklearn.svm import SVC
svm = SVC()

In [199]:
model_pipeline(svm)

0 번째 accuracy non_scale : 0.9693% StandardScale : 0.9858% MinMax : 0.8986% MaxAbs : 0.9599%
1 번째 accuracy non_scale : 0.9764% StandardScale : 0.9882% MinMax : 0.8889% MaxAbs : 0.9669%
2 번째 accuracy non_scale : 0.9551% StandardScale : 0.9787% MinMax : 0.8629% MaxAbs : 0.9504%
3 번째 accuracy non_scale : 0.9668% StandardScale : 0.9882% MinMax : 0.8815% MaxAbs : 0.9597%
4 번째 accuracy non_scale : 0.9692% StandardScale : 0.9810% MinMax : 0.9005% MaxAbs : 0.9645%

 mean accuracy non_scale : 0.9674% StandardScale : 0.9844% MinMax : 0.8865% MaxAbs : 0.9603%


In [215]:
pipeline1 = Pipeline([('StandardScaler', standardsc), ('svm', svm)])
pipeline1.fit(X,y)
get_eval(urine['y'],pipeline1.predict(urine.drop('y', axis = 1)))

accuracy : 0.9683 precision : 1.0000 recall : 0.9524 F1 : 0.9756 AUC : 0.9762
[[21  0]
 [ 2 40]]
